In [1]:
from amnesiac import blurry_memory
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import string

In [2]:
random.seed(10)
passcode_length = 10
passcode_options = list(string.ascii_uppercase+string.digits+'_')
np.random.shuffle(passcode_options)
print(passcode_options)

['Z', '8', 'E', '3', '7', 'F', 'H', '5', '_', '2', 'T', 'L', 'B', '0', 'W', 'O', 'N', '4', 'A', 'G', 'V', '6', 'I', '9', 'Y', 'J', 'R', 'D', 'M', '1', 'C', 'Q', 'P', 'S', 'K', 'U', 'X']


In [3]:
population_size = 100
num_parents = 20

In [4]:
population = [random.choices(passcode_options,k=passcode_length) for i in range(population_size)]

In [5]:
def fitness(population,user_id,passwd_no):
    return [[i,int(blurry_memory([''.join(i)],user_id,passwd_no)[''.join(i)]*10)] for i in population]

In [6]:
def select_parents(fitness_scores,method='rank'):
    if method == 'rank':
        return [i[0] for i in sorted(fitness_scores, key=lambda x: x[1], reverse = True)[:num_parents]]
    if method == 'mixed_rank':
        fitness_scores_sorted = sorted(fitness_scores, key=lambda x: x[1], reverse = True)
        parents = [i[0] for i in fitness_scores_sorted]
        selected_parents = parents[:int(num_parents*0.6)]+parents[-int(num_parents*0.2):]
        return random.choices(parents[int(num_parents*0.6):-int(num_parents*0.2):],k=int(num_parents*0.2)) + selected_parents
    if method == 'tournament':
        parents_list = []
        while len(parents_list) <= num_parents:
            best_random_tounamenet_players = random.choices(fitness_score,k=3)
            best_random_tounamenet_players = sorted(best_random_tounamenet_players, key=lambda x: x[1], reverse = True)
            parents_list.append(best_random_tounamenet_players[0][0])
            fitness_score.remove(best_random_tounamenet_players[0])
        return parents_list

In [7]:
# crossover logic
def crossover(parent1,parent2):
    [startGene,endGene] = sorted(random.choices(range(passcode_length),k=2))
    if random.random()>0.5:
        return parent1[:startGene]+parent2[startGene:endGene]+parent1[endGene:]
    else:
        return parent2[:startGene]+parent1[startGene:endGene]+parent2[endGene:]


# reproduction
def create_children(all_parents):
    children = []
    for i in range(len(population)):
        parent1 = all_parents[int(random.random() * len(all_parents))]
        parent2 = all_parents[int(random.random() * len(all_parents))]
        children.append(crossover(parent1,parent2))
    return children

In [8]:
def mutation(children_set):
    for child in children_set:
        if random.random() < 0.2:
            #mutate if rate is less 20 percent 
            child[int(random.random() * passcode_length)] = random.choice(passcode_options)
    return children_set

In [9]:
fitness_tracker = []
fitness_scores_list = []
generations = 0
t0 = time.time()
while True:
    fitness_scores = fitness(population,190573735,1)
    fitness_scores_list.append(fitness_scores)
#     fitness_tracker.append(max([i[1] for i in fitness_scores]))
#     solutions.append(''.join([i[0] for i in fitness_scores if i[1] == max([i[1] for i in fitness_scores])][0]))
#     print(''.join([i[0] for i in fitness_scores if i[1] == max([i[1] for i in fitness_scores])][0]))
    if max([i[1] for i in fitness_scores]) == passcode_length:
        print("Cracked in {} generations, and {} seconds! \nDiscovered passcode = {}".format(generations,time.time() - t0,[i[0] for i in fitness_scores if i[1] == passcode_length][0]))
        break
    parents = select_parents(fitness_scores,method='roulette')
    children = create_children(parents)
    population = mutation(children)
    generations += 1
#     print(generations)

KeyboardInterrupt: 

In [ ]:
# fitness_tracker.append(max([j[1] for i in fitness_scores_list for j in i]))

fitness_tracker = [max([i[1] for i in fitness_scores_list[j]]) for j in range(len(fitness_scores_list))]
    
        
fig = plt.figure()
plt.plot(list(range(generations+1)), fitness_tracker)
fig.suptitle('Fitness Score by Generation', fontsize=14, fontweight='bold')
ax = fig.add_subplot(111)
ax.set_xlabel('Generation')
ax.set_ylabel('Fitness Score')
plt.show()